# Streaming Design Patterns

The Lakehouse has been designed from the beginning to work seamlessly with datasets that grow infinitely over time. While Spark Structured Streaming is often positioned as a near real-time data processing solution, it combines with Delta Lake to also provide easy batch processing of incremental data while drastically simplifying the overhead required to track data changes over time.

## Learning Objectives
By the end of this lessons, student will be able to:
- Use Structured Streaming to complete simple incremental ETL
- Perform incremental writes to multiple tables
- Incrementally update values in a key value store
- Process Change Data Capture (CDC) data into Delta Tables using **`MERGE`**
- Join two incremental tables
- Join incremental and batch tables

Run the following script to setup necessary variables and clear out past runs of this notebook.

In [0]:
%run ../Includes/Classroom-Setup-2.1

## Simple Incremental ETL

Likely the highest volume of data being processed by most organizations could largely be describing as moving data from one location to another while applying light transformations and validations. 

As most source data continues to grow as time passes, it's appropriate to refer to this data as incremental (sometimes also referred to as streaming data). 

Structured Streaming and Delta Lake make incremental ETL easy. 

Below we'll create a simple table and insert some values.

In [0]:
%sql

CREATE TABLE bronze 
(id INT, name STRING, value DOUBLE); 

INSERT INTO bronze
VALUES (1, "Yve", 1.0),
       (2, "Omar", 2.5),
       (3, "Elia", 3.3)

The following cell defines an incremental read on the table just created using Structured Streaming, adds a field to capture when the record was processed, and writes out to a new table as a single batch.

In [0]:
from pyspark.sql import functions as F

def update_silver():
    query = (spark.readStream
                  .table("bronze")
                  .withColumn("processed_time", F.current_timestamp())
                  .writeStream.option("checkpointLocation", f"{DA.paths.checkpoints}/silver")
                  .trigger(availableNow=True)
                  .table("silver"))
    
    query.awaitTermination()


While this code uses Structured Streaming, it's appropriate to think of this as a triggered batch processing incremental changes.

<img src="https://files.training.databricks.com/images/icon_note_32.png"></img>
To facilitate the demonstration of structured streams, we are using **`trigger(availableNow=True)`** to slow
down the processing of the data combined with **`query.awaitTermination()`** to prevent the lesson from
moving forward until the one batch is processed.  **trigger-available-now** is very similar to **trigger-once** but can run
multiple batches until all available data is consumed instead of once big batch and is introduced in
<a href="https://spark.apache.org/releases/spark-release-3-3-0.html" target="_blank">Spark 3.3.0</a> and
<a href="https://docs.databricks.com/release-notes/runtime/10.4.html" target="_blank">Databricks Runtime 10.4 LTS</a>.

In [0]:
update_silver()

As expected, the stream runs for a very brief time, and the **`silver`** table written contains all the values previously written to **`bronze`**.

In [0]:
%sql
SELECT * FROM silver
ORDER BY processed_time DESC, id DESC

Processing new records is as easy as adding them to our source table **`bronze`**...

In [0]:
%sql
INSERT INTO bronze
VALUES (4, "Ted", 4.7),
       (5, "Tiffany", 5.5),
       (6, "Vini", 6.3)

... and re-executing the incremental batch processing code.

In [0]:
update_silver()

Delta Lake is ideally suited for easily tracking and propagating inserted data through a series of tables. This pattern has a number of names, including "medallion", "multi-hop", "Delta", and "bronze/silver/gold" architecture.

In [0]:
%sql
SELECT * FROM silver
ORDER BY processed_time DESC, id DESC

## Writing to Multiple Tables

Those familiar with Structured Streaming may be aware that the **`foreachBatch`** method provides the option to execute custom data writing logic on each microbatch of streaming data.

The Databricks Runtime provides guarantees that these <a href="https://docs.databricks.com/delta/delta-streaming.html#idempot-write" target="_blank">streaming Delta Lake writes will be idempotent</a>, even when writing to multiple tables, IF you set the "txnVersion" and "txnAppId" options. This is especially useful when data for multiple tables might be contained within a single record.  This was added in <a href="https://docs.databricks.com/release-notes/runtime/8.4.html" target="_blank">Databricks Runtime 8.4</a>.

The code below first defines the custom writer logic to append records to two new tables, and then demonstrates using this function within **`foreachBatch`**.

There is some debate as to whether you should use foreachBatch to write to multiple tables or to simply use multiple streams.  Generally multiple streams is the simpler and more efficient design because it allows streaming jobs writing to each table to run independently of each other.  Whereas using foreachBatch to write to multiple tables has the advantage of keeping writes to the two tables in sync.

In [0]:
def write_twice(microBatchDF, batchId):
    appId = "write_twice"
    
    microBatchDF.select("id", "name", F.current_timestamp().alias("processed_time")).write.option("txnVersion", batchId).option("txnAppId", appId).mode("append").saveAsTable("silver_name")
    
    microBatchDF.select("id", "value", F.current_timestamp().alias("processed_time")).write.option("txnVersion", batchId).option("txnAppId", appId).mode("append").saveAsTable("silver_value")


def split_stream():
    query = (spark.readStream.table("bronze")
                 .writeStream
                 .foreachBatch(write_twice)
                 .option("checkpointLocation", f"{DA.paths.checkpoints}/split_stream")
                 .trigger(availableNow=True)
                 .start())
    
    query.awaitTermination()
    

Note that while a stream will again be triggered, the two writes contained within the **`write_twice`** function are using Spark batch syntax. This will always be the case for writers called by **`foreachBatch`**.

In [0]:
split_stream()

The cells below demonstrate the logic was applied properly to split the initial data into two tables.

In [0]:
%sql
SELECT * FROM silver_name
ORDER BY processed_time DESC, id DESC

Note that the **`processed_time`** for each of these tables differs slightly. The logic defined above captures the current timestamp at the time each write executes, demonstrating that while both writes happen within the same streaming microbatch process, they are fully independent transactions (as such, downstream logic should be tolerant for slightly asynchronous updates).

In [0]:
%sql
SELECT * FROM silver_value
ORDER BY processed_time DESC, id DESC

Insert more values into the **`bronze`** table.

In [0]:
%sql
INSERT INTO bronze
VALUES (7, "Viktor", 7.4),
       (8, "Hiro", 8.2),
       (9, "Shana", 9.9)

And we can now pick up these new records and write to two tables.

In [0]:
split_stream()

As expected, only new values are inserted into the two tables, again a few moments apart.

In [0]:
%sql
SELECT * FROM silver_name
ORDER BY processed_time DESC, id DESC

In [0]:
%sql
SELECT * FROM silver_value
ORDER BY processed_time DESC, id DESC

## Update Aggregates in a Key-Value Store

Incremental aggregation can be useful for a number of purposes, including dashboarding and enriching reports with current summary data.

The logic below defines a handful of aggregations against the **`silver`** table.

In [0]:
def update_key_value():
    query = (spark.readStream
                  .table("silver")
                  .groupBy("id")
                  .agg(F.sum("value").alias("total_value"), 
                       F.mean("value").alias("avg_value"),
                       F.count("value").alias("record_count"))
                  .writeStream
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/key_value")
                  .outputMode("complete")
                  .trigger(availableNow=True)
                  .table("key_value"))
    
    query.awaitTermination()
    

**NOTE**: Because the transformations above require shuffling data, setting the number of shuffle partitions to the maximum number of cores will provide more efficient performance. 

The default number of shuffle partitions (200) can cripple many streaming jobs.

As such, it's a reasonably good practice to simply use the maximum number of cores as the high end, and if smaller, maintain a factor of the number of cores.

Naturally, this generalized advice changes as you increase the number of streams running on a single cluster.

In [0]:
print(f"Executor cores: {sc.defaultParallelism}")
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

In [0]:
update_key_value()

In [0]:
%sql
SELECT * FROM key_value
ORDER BY id

Adding more values to the **`silver`** table will allow more interesting aggregation.

In [0]:
%sql
INSERT INTO silver
VALUES (1, "Yve", 1.0, current_timestamp()),
       (2, "Omar", 2.5, current_timestamp()),
       (3, "Elia", 3.3, current_timestamp()),
       (7, "Viktor", 7.4, current_timestamp()),
       (8, "Hiro", 8.2, current_timestamp()),
       (9, "Shana", 9.9, current_timestamp())

One thing to note is that the logic being executed is currently overwriting the resulting table with each write. In the next section, **`MERGE`** will be used in combination with **`foreachBatch`** to update existing records. This pattern can also be applied with key-value stores.

In [0]:
update_key_value()

In [0]:
%sql
SELECT * FROM key_value
ORDER BY id

## Processing Change Data Capture Data
While the change data capture (CDC) data emitted by various systems will vary greatly, incrementally processing these data with Databricks is straightforward.

Here the **`bronze_status`** table will represent the raw CDC information, rather than row-level data.

In [0]:
%sql
CREATE TABLE bronze_status 
(user_id INT, status STRING, update_type STRING, processed_timestamp TIMESTAMP);

INSERT INTO bronze_status
VALUES  (1, "new", "insert", current_timestamp()),
        (2, "repeat", "update", current_timestamp()),
        (3, "at risk", "update", current_timestamp()),
        (4, "churned", "update", current_timestamp()),
        (5, null, "delete", current_timestamp())

The **`silver_status`** table below has been created to track the current **`status`** for a given **`user_id`**.

In [0]:
%sql
CREATE TABLE silver_status (user_id INT, status STRING, updated_timestamp TIMESTAMP)

The **`MERGE`** statement can easily be written with SQL to apply CDC changes appropriately, given the type of update received.

The rest of the **`upsert_cdc`** method contains the logic necessary to run SQL code against a micro-batch in a PySpark DataStreamWriter.

In [0]:
def upsert_cdc(microBatchDF, batchID):
    microBatchDF.createTempView("bronze_batch")
    
    query = """
        MERGE INTO silver_status s
        USING bronze_batch b
        ON b.user_id = s.user_id
        WHEN MATCHED AND b.update_type = "update"
          THEN UPDATE SET user_id=b.user_id, status=b.status, updated_timestamp=b.processed_timestamp
        WHEN MATCHED AND b.update_type = "delete"
          THEN DELETE
        WHEN NOT MATCHED AND b.update_type = "update" OR b.update_type = "insert"
          THEN INSERT (user_id, status, updated_timestamp)
          VALUES (b.user_id, b.status, b.processed_timestamp)
    """
    
    microBatchDF._jdf.sparkSession().sql(query)
    
def streaming_merge():
    query = (spark.readStream
                  .table("bronze_status")
                  .writeStream
                  .foreachBatch(upsert_cdc)
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/silver_status")
                  .trigger(availableNow=True)
                  .start())
    
    query.awaitTermination()
    

As always, we incrementally process newly arriving records.

In [0]:
streaming_merge()

In [0]:
%sql
SELECT * FROM silver_status
ORDER BY updated_timestamp DESC, user_id DESC

Inserting new records will allow us to then apply these changes to our silver data.

In [0]:
%sql
INSERT INTO bronze_status
VALUES  (1, "repeat", "update", current_timestamp()),
        (2, "at risk", "update", current_timestamp()),
        (3, "churned", "update", current_timestamp()),
        (4, null, "delete", current_timestamp()),
        (6, "new", "insert", current_timestamp())

In [0]:
streaming_merge()

Note that at present, the logic would not be particularly robust to data arriving out-of-order or duplicate records (but these occurences can be handled).

In [0]:
%sql
SELECT * FROM silver_status
ORDER BY updated_timestamp DESC, user_id DESC

## Joining Two Incremental Tables

Note that there are many intricacies around watermarking and windows when dealing with incremental joins, and that not all join types are supported.

In [0]:
def stream_stream_join():
    nameDF = spark.readStream.table("silver_name")
    valueDF = spark.readStream.table("silver_value")
    
    return (nameDF.join(valueDF, nameDF.id == valueDF.id, "inner")
                  .select(nameDF.id, 
                          nameDF.name, 
                          valueDF.value, 
                          F.current_timestamp().alias("joined_timestamp"))
                  .writeStream
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/joined")
                  .queryName("joined_streams_query")
                  .table("joined_streams")
           )

Note that the logic defined above does not set a **`trigger`** option.

This means that the stream will run in continuous execution mode, triggering every 500ms by default.

In [0]:
query = stream_stream_join()

This also means that it is possible to read the new tables before there is any data in them.

Because the stream never stops we can't block until the trigger-available-now stream has terminated with **`awaitTermination()`**.

Instead we can block until "some" data is processed by leveraging **`query.recentProgress`**.

In [0]:
def block_until_stream_is_ready(query, min_batches=2):
    import time
    while len(query.recentProgress) < min_batches:
        time.sleep(5) # Give it a couple of seconds

    print(f"The stream has processed {len(query.recentProgress)} batchs")
    
block_until_stream_is_ready(query)

Running **`display()`** on a streaming table is a way to monitor table updates in near-real-time while in interactive development.

In [0]:
display(spark.readStream.table("joined_streams"))

<img src="https://files.training.databricks.com/images/icon_note_32.png"> Anytime a streaming read is displayed to a notebook, a streaming job will begin.

Here a second stream is started.  One is processing the data as part of our original pipline, and now a second streaming job is running to update the **`display()`** function with the latest results.

In [0]:
for stream in spark.streams.active:
    print(stream.name)

Here we'll add new values to the **`bronze`** table.

In [0]:
%sql
INSERT INTO bronze
VALUES (10, "Pedro", 10.5),
       (11, "Amelia", 11.5),
       (12, "Diya", 12.3),
       (13, "Li", 13.4),
       (14, "Daiyu", 14.2),
       (15, "Jacques", 15.9)

The stream-stream join is configured against the tables resulting from the **`split_stream`** function; run this again and data should quickly process through the streaming join running above.

In [0]:
split_stream()

Interactive streams should always be stopped before leaving a notebook session, as they can keep clusters from auto-terminating and incur unnecessary cloud costs.  
You can terminate a streaming job by clicking "Cancel" on a running cell, "Stop Execution" at the top of the notebook, or by running the code below.

In [0]:
for stream in spark.streams.active:
    print(f"Stopping {stream.name}")
    stream.stop()
    stream.awaitTermination()

## Join Incremental and Static Data

While incremental tables are ever-appending, static tables typically can be thought of as containing data that may be changed or overwritten.

Because of Delta Lake's transactional guarantees and caching, Databricks ensures that each microbatch of streaming data that's joined back to a static table will contain the current version of data from the static table.

In [0]:
statusDF = spark.read.table("silver_status")
bronzeDF = spark.readStream.table("bronze")

query = (bronzeDF.alias("bronze")
                 .join(statusDF.alias("status"), bronzeDF.id==statusDF.user_id, "inner")
                 .select("bronze.*", "status.status")
                 .writeStream
                 .option("checkpointLocation", f"{DA.paths.checkpoints}/join_status")
                 .queryName("joined_status_query")
                 .table("joined_status")
)

Again, wait until we have some data before moving forward

In [0]:
block_until_stream_is_ready(query)

In [0]:
%sql
SELECT * FROM joined_status
ORDER BY id DESC

Only those records with a matching **`id`** in **`joined_status`** at the time the stream is processed will be represented in the resulting table.

In [0]:
%sql
SELECT * FROM silver_status
ORDER BY updated_timestamp DESC, user_id DESC

Processing new records into the **`silver_status`** table will not automatically trigger updates to the results of the stream-static join.

In [0]:
%sql
INSERT INTO bronze_status
VALUES  (11, "repeat", "update", current_timestamp()),
        (12, "at risk", "update", current_timestamp()),
        (16, "new", "insert", current_timestamp()),
        (17, "repeat", "update", current_timestamp())

In [0]:
streaming_merge()

In [0]:
%sql
SELECT * FROM joined_status
ORDER BY id DESC

Only new data appearing on the streaming side of the query will trigger records to process using this pattern.

In [0]:
%sql
INSERT INTO bronze
VALUES (16, "Marissa", 1.9),
       (17, "Anne", 2.7)

The incremental data in a stream-static join "drives" the stream, guaranteeing that each microbatch of data joins with the current values present in the valid version of the static table.

In [0]:
%sql
SELECT * FROM joined_status
ORDER BY id DESC

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()